In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from dataclasses import dataclass
import io
import json
import os
from pathlib import Path
from pprint import pprint
import re
import requests
import sys
from typing import Optional

if '..' not in sys.path: sys.path.append('..')

from datasets import load_dataset
import numpy as np
import pandas as pd
from pydantic_yaml import parse_yaml_file_as, to_yaml_file
import torch
from torch import nn
import torch.nn.functional as F
from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel, BertTokenizer, AutoTokenizer
from transformers.modeling_outputs import Seq2SeqLMOutput, BaseModelOutputWithPastAndCrossAttentions, CausalLMOutputWithCrossAttentions

from mllm.config.model import GenmixTrainDsType, TokensAggType, GenmixembBertCfg, copy_override_genmixemb_bert_cfg, \
    gen_prefpostfix_genmixemb_bert
from mllm.exp.args import GENMIXEMB_BERT_MODEL_CFG_FNAME, create_bool_str_field, is_arg_true
from mllm.model.genmixemb_bert import GenmixembBert
from mllm.train.mask_utils import MaskCfg
from mllm.train.utils import find_create_train_path, log_weights_grads_stats, SumTuple, QnaTuple
from mllm.data.wiki.itwiki import WikiItem, get_wiki_batch_iterators, WikiBatch
from mllm.utils.utils import rethrow
from mllm.data.utils import get_squadv2_df


# BERT Generator model inference
## Configs and paths

In [11]:
DATA_PATH = Path(os.path.expandvars('$HOME')) / 'data'

bert_model_name = 'bert-base-uncased'
random_seed = 111
train_genmixemb_bert_path = DATA_PATH / 'train_mllm_genmixemb_bert'
genmixemb_subdir = 'genmixemb-20250713_202718-bertbaseuncased-d768-mxo50-aggBrt-sub0-dsWki-tmax100-tragF'
genmixemb_subdir = 'genmixemb-20250715_035750-bertbaseuncased-d768-mxo50-aggBrt-sub2-dsWki-tmax100-tragT'
genmixemb_subdir = 'genmixemb-20250716_213252-bertbaseuncased-d768-mxo50-aggBrt-sub0-dsWki-tmax100-msk_sep_0.5/0.15_seq_0.5/0.2/20-tragF'
genmixemb_subdir = 'genmixemb-20250718_220105-bertbaseuncased-d768-mxo50-aggBrt-sub2-dsWki-tmax100-tragT'
genmixemb_subdir = 'genmixemb-20250721_083250-bertbaseuncased-d768-mxo50-aggPyr-agtDecim-stp0-lvl1-lrs2-dsWki-tmax256-tragF-nxtsnt'
genmixemb_subdir = 'genmixemb-20250721_212402-bertbaseuncased-d768-mxo50-aggPyr-agtDecim-stp2-lvl2-lrs2-dsWki-tmax512-tragT-nxtsnt'
genmixemb_subdir = 'genmixemb-20250722_213424-bertbaseuncased-d768-mxo50-aggPyr-agtDecim-stp2-lvl3-lrs2-dsWki-tmax512-tragT-nxtsnt'

train_genmixemb_bert_path = DATA_PATH / 'train_mllm_genmixembbert_qna'
genmixemb_subdir = 'genmixemb-20250726_122548-bertbaseuncased-d768-mxo50-aggPyr-agtDecim-stp0-lvl3-lrs2-dsQna-tmax384-tragF'

genmixemb_train_path = train_genmixemb_bert_path / genmixemb_subdir
genmixemb_snapshot_fpath = genmixemb_train_path / 'best.pth'

device_name = 'cpu'
# device_name = 'cuda'

device = torch.device(device_name)
print('Device:', device)

parts = genmixemb_subdir.split('-')
n_toks_max = 0
mask_cfg = None
pred_next_sent = False
for part in parts:
    if part.startswith('tmax'):
        n_toks_max = int(part[4:])
    elif part.startswith('msk_'):
        subparts = part.split('_')
        # postfix_parts.append(f'msk_sep_{sep_freq}/{sep_frac}_seq_{seq_freq}/{seq_max_frac}/{mask_cfg.seq_max_len}')
        sep_part, seq_part = subparts[2], subparts[4]
        sep_freq, sep_frac = sep_part.split('/')
        seq_freq, seq_max_frac, seq_max_len = seq_part.split('/')
        sep_freq, sep_frac = float(sep_freq), float(sep_frac)
        seq_freq, seq_max_frac, seq_max_len = float(seq_freq), float(seq_max_frac), int(seq_max_len)
        mask_cfg = MaskCfg(
            sep_freq=sep_freq, sep_frac=sep_frac, seq_freq=seq_freq, seq_max_frac=seq_max_frac,
            seq_max_len=seq_max_len,
        )
    elif part == 'nxtsnt':
        pred_next_sent = True

print(f'n_toks_max = {n_toks_max}')
print('Mask cfg:', mask_cfg)
print(f'pred_next_sent = {pred_next_sent}')

batch_size = 5


Device: cpu
n_toks_max = 384
Mask cfg: None
pred_next_sent = False


In [12]:
model_cfg = parse_yaml_file_as(GenmixembBertCfg, genmixemb_train_path / GENMIXEMB_BERT_MODEL_CFG_FNAME)
pprint(model_cfg.dict())

{'bert_agg_n_subseq_toks': 2,
 'bert_model_name': 'bert-base-uncased',
 'd_model': 768,
 'max_out_toks': 50,
 'pyr_agg_n_layers_per_level': 2,
 'pyr_agg_n_levels': 3,
 'pyr_agg_step': 0,
 'pyr_agg_type': <HgReductType.Decim: 'decim'>,
 'toks_agg_type': <TokensAggType.Pyramid: 'pyr'>,
 'train_agg_model': False}


## Load models and dataset
### Model

In [13]:
model = GenmixembBert(model_cfg, device=device)
tkz = model.tkz

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of BertGenerationDecoder were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossatte

In [14]:
print(f'Load {genmixemb_snapshot_fpath}')
checkpoint = torch.load(genmixemb_snapshot_fpath, map_location=device)
model.load_state_dict(checkpoint['model'], strict=True)
del checkpoint
model.eval()
None

Load /home/misha/data/train_mllm_genmixembbert_qna/genmixemb-20250726_122548-bertbaseuncased-d768-mxo50-aggPyr-agtDecim-stp0-lvl3-lrs2-dsQna-tmax384-tragF/best.pth


## Wiki dataset
### Loading

In [112]:
wiki_ds_name, wiki_ds_subdir = '20220301.en', 'wikipedia'
dss_wiki = load_dataset(wiki_ds_subdir, wiki_ds_name, cache_dir=str(DATA_PATH))
ds_wiki = dss_wiki['train']
n_docs = len(ds_wiki)
print(f'Wikipedia {wiki_ds_name} docs: {n_docs}')

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

Wikipedia 20220301.en docs: 6458670


In [123]:
def get_wiki_batch(i_batch: int, batch_size: int = batch_size) -> WikiBatch:
    i1 = i_batch * batch_size
    i2 = i1 + batch_size
    items = []
    for i in range(i1, i2):
        row = ds_wiki[i]
        wiki_item = WikiItem(
            tkz=tkz, ind=i, title=row['title'], text=row['text'], max_len=n_toks_max, mask_cfg=mask_cfg, pred_next_sent=pred_next_sent,
            max_pred_len=model_cfg.max_out_toks,
        )
        items.append(wiki_item)
    batch = WikiBatch(items=items, device=device)
    return batch

### Wiki inference

In [164]:
i_batch = 0
batch = get_wiki_batch(i_batch)
# [n_batch, max_len]
b_toks, b_masked_toks, b_mask, b_tgt_toks = batch.get_tensors()

Token indices sequence length is longer than the specified maximum sequence length for this model (531 > 512). Running this sequence through the model will result in indexing errors


In [165]:
with_mask = False
# with_mask = True
for i in range(batch_size):
    toks, masked_toks, mask = b_toks[i], b_masked_toks[i], b_mask[i]
    tgt_toks = None
    if b_tgt_toks is not None:
        tgt_toks = b_tgt_toks[i]
    toks_inp = masked_toks if with_mask else toks
    toks_out = model.gen_on_wiki(toks=toks_inp)
    toks_out = toks_out.squeeze(0)
    s_inp = tkz.decode(toks_inp)
    s_out = tkz.decode(toks_out)
    if with_mask:
        s_src = tkz.decode(toks)
        print(f'{i:03d}. Msk: {s_src}')
    print(f'{i:03d}. Inp: {s_inp}')
    if tgt_toks is not None:
        s_tgt = tkz.decode(tgt_toks)
        print(f'{i:03d}. Tgt: {s_tgt}')
    print(f'{i:03d}. Out: {s_out}')
    # break

000. Inp: , among them respect for child autonomy and relying on reasoning rather than indoctrination as a teaching method, have spread among mainstream educational institutions. judith suissa names three schools as explicitly anarchists schools, namely the free skool santa cruz in the united states which is part of a wider american - canadian network of schools, the self - managed learning college in brighton, england, and the paideia school in spain. anarchism and the state objection to the state and its institutions is a sine qua non of anarchism. anarchists consider the state as a tool of domination and believe it to be illegitimate regardless of its political tendencies. instead of people being able to control the aspects of their life, major decisions are taken by a small elite. authority ultimately rests solely on power, regardless of whether that power is open or transparent, as it still has the ability to coerce people. another anarchist argument against states is that the peo

In [67]:
(b_masked_toks == tkz.mask_token_id).sum()

tensor(100)

### Custom text

In [172]:
context = '''
Lithuania,[b] officially the Republic of Lithuania,[c] is a country in the Baltic region of Europe.[d] It is one of three Baltic states and lies on the eastern shore of the Baltic Sea, bordered by Latvia to the north, Belarus to the east and south, Poland to the south, and the Russian semi-exclave of Kaliningrad Oblast to the southwest, with a maritime border with Sweden to the west. Lithuania covers an area of 65,300 km2 (25,200 sq mi), with a population of 2.89 million. Its capital and largest city is Vilnius; other major cities include Kaunas, Klaipėda, Šiauliai and Panevėžys. Lithuanians who are the titular nation and form the majority of the country's population, belong to the ethnolinguistic group of Balts and speak Lithuanian. For millennia, the southeastern shores of the Baltic Sea were inhabited by various Baltic tribes. In the 1230s, Lithuanian lands were united for the first time by Mindaugas, who formed the Kingdom of Lithuania on 6 July 1253. Subsequent expansion and consolidation resulted in the Grand Duchy of Lithuania, which by the 14th century was the largest country in Europe. In 1386, the Grand Duchy entered into a de facto personal union with the Crown of the Kingdom of Poland. The two realms were united into the bi-confederal Polish-Lithuanian Commonwealth in 1569, forming one of the largest and most prosperous states in Europe. The Commonwealth lasted more than two centuries, until neighbouring countries gradually dismantled it between 1772 and 1795, with the Russian Empire annexing most of Lithuania's territory. Towards the end of World War I, Lithuania declared independence in 1918, founding the modern Republic of Lithuania. In World War II, Lithuania was occupied by the Soviet Union, then by Nazi Germany, before being reoccupied by the Soviets in 1944. Lithuanian armed resistance to the Soviet occupation lasted until the early 1950s. On 11 March 1990, a year before the formal dissolution of the Soviet Union, Lithuania became the first Soviet republic to break away when it proclaimed the restoration of its independence. Lithuania is a developed country with a high-income and an advanced economy ranking very high in Human Development Index.'''
context2 = 'Lithuania ranks highly in digital infrastructure,[25][26] press freedom and happiness.[27] Lithuania is a member of the United Nations, the European Union, the Council of Europe, the Council of the Baltic Sea States, the Eurozone, the Nordic Investment Bank, the International Monetary Fund, the Schengen Agreement, NATO, OECD and the World Trade Organization. It also participates in the Nordic-Baltic Eight (NB8) regional co-operation format.'

question = 'When Lithuania declared its independence and from whom?'
question = 'What is a capital of Lithuania?'
text = f'''
Context: {context.strip()}. Question: {question}. Answer: 
'''
text = f'''
<context>{context.strip()}</context> <question>{question}</question> <answer> The capital of Lithuania is
'''
text = text.strip()
title = 'Lithuania'

In [173]:
toks_in = tkz(text).input_ids
toks_in = torch.tensor(toks_in, dtype=torch.long, device=device)
toks_out = model.gen_on_wiki(toks_in)
s_out = tkz.decode(toks_out.squeeze())
s_out

'[CLS] in addition, the central economic development commission ( cei ) of the european union, which was established in 1946, is a separate independent state. these countries are not a separate state, but an entirely administrative unit of the voivodeship, the administrative region and'

In [171]:
tkz.decode(toks_in)

"[CLS] < context > lithuania, [ b ] officially the republic of lithuania, [ c ] is a country in the baltic region of europe. [ d ] it is one of three baltic states and lies on the eastern shore of the baltic sea, bordered by latvia to the north, belarus to the east and south, poland to the south, and the russian semi - exclave of kaliningrad oblast to the southwest, with a maritime border with sweden to the west. lithuania covers an area of 65, 300 km2 ( 25, 200 sq mi ), with a population of 2. 89 million. its capital and largest city is vilnius ; other major cities include kaunas, klaipeda, siauliai and panevezys. lithuanians who are the titular nation and form the majority of the country's population, belong to the ethnolinguistic group of balts and speak lithuanian. for millennia, the southeastern shores of the baltic sea were inhabited by various baltic tribes. in the 1230s, lithuanian lands were united for the first time by mindaugas, who formed the kingdom of lithuania on 6 july 

### Try nltk for sentence splitting

In [96]:
import nltk
from nltk.tokenize import sent_tokenize
from timeit import default_timer as timer

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/misha/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [101]:
item = ds_wiki[0]
txt = item['text']
s = txt.split('\n')[0]
print(len(txt), len(s), s)

43985 559 Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.


In [97]:
sents = sent_tokenize(s)
for sent in sents:
    print(sent)

Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy.
Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful.
As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.


In [100]:
N = 1000
t1 = timer()
for i in range(N):
    sents = sent_tokenize(txt)
t2 = timer()
delta = (t2 - t1) / N
print(f'sent_tokenize avg time: {delta:0.6f}')


sent_tokenize avg time: 0.005308


## Squad v2 dataset
### Loading

In [16]:
df_squad = get_squadv2_df(exclude_empty_answers=True)
print(len(df_squad))
print(df_squad.head())

Remove empty answers from dataset squad_v2. Size: 142192 --> 92749
92749
                              id             title  \
22029   56f7469ca6d7ea1400e17137   Classical_music   
102704  5731c705e17f3d14004223cd  Steven_Spielberg   
119773  57314900a5e9cc1400cdbe0f      Qing_dynasty   
29865   5706336875f01819005e7a87               MP3   
50008   5724f8f90a492a190043569c       Freemasonry   

                                                  context  \
22029   During the Baroque era, keyboard music played ...   
102704  Apart from being an ardent gamer Spielberg has...   
119773  The dynasty lost control of peripheral territo...   
29865   Due to the tree structure of the filter bank, ...   
50008   In 1799, English Freemasonry almost came to a ...   

                                                 question  \
22029       The violin family is what type of instrument?   
102704  Who wrote the script for 'Someone's in the Kit...   
119773              What triggered the Tientsin Mass

In [20]:
from mllm.data.itsquadv2 import QnaItemV2, QnaBatchV2


def get_qna_batch(i_batch: int, batch_size: int = batch_size) -> QnaBatchV2:
    i1 = i_batch * batch_size
    i2 = i1 + batch_size
    items = []
    for i in range(i1, i2):
        row = df_squad.iloc[i]
        answer = row.answers['text'][0]
        qna_item = QnaItemV2(
            tkz=tkz, ind=i, context=row.context, question=row.question, answer=answer,
        )
        items.append(qna_item)
    batch = QnaBatchV2(
        items=items, max_inp_len=n_toks_max, max_out_len=model_cfg.max_out_toks, device=device,
    )
    return batch

### QnA inference

In [21]:
i_batch = 0
batch = get_qna_batch(i_batch)
# [n_batch, max_len]
b_ctx_toks, b_que_toks, b_ans_toks, b_cq_toks = batch.get_tensors()

In [22]:
for i in range(batch_size):
    ctx_toks, que_toks, ans_toks, cq_toks = b_ctx_toks[i], b_que_toks[i], b_ans_toks[i], b_cq_toks[i]
    tgt_toks = ans_toks
    toks_inp = cq_toks
    toks_out = model.gen_on_wiki(toks=toks_inp)
    toks_out = toks_out.squeeze(0)
    s_inp = tkz.decode(toks_inp)
    s_out = tkz.decode(toks_out)
    s_tgt = tkz.decode(tgt_toks)
    print(f'{i:03d}. Inp: {s_inp}')
    print(f'{i:03d}. Tgt: {s_tgt}')
    print(f'{i:03d}. Out: {s_out}')
    # break

`generation_config` default values have been modified to match model-specific defaults: {'pad_token_id': 0}. If this is not desired, please set these values explicitly.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


000. Inp: [CLS] during the baroque era, keyboard music played on the harpsichord and pipe organ became increasingly popular, and the violin family of stringed instruments took the form generally seen today. opera as a staged musical drama began to differentiate itself from earlier musical and dramatic forms, and vocal forms like the cantata and oratorio became more common. vocalists began adding embellishments to melodies. instrumental ensembles began to distinguish and standardize by size, giving rise to the early orchestra for larger ensembles, with chamber music being written for smaller groups of instruments where parts are played by individual ( instead of massed ) instruments. the concerto as a vehicle for solo performance accompanied by an orchestra became widespread, although the relationship between soloist and orchestra was relatively simple. [SEP] the violin family is what type of instrument? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

### Custom text

In [31]:
context = '''
Lithuania,[b] officially the Republic of Lithuania,[c] is a country in the Baltic region of Europe.[d] It is one of three Baltic states and lies on the eastern shore of the Baltic Sea, bordered by Latvia to the north, Belarus to the east and south, Poland to the south, and the Russian semi-exclave of Kaliningrad Oblast to the southwest, with a maritime border with Sweden to the west. Lithuania covers an area of 65,300 km2 (25,200 sq mi), with a population of 2.89 million. Its capital and largest city is Vilnius; other major cities include Kaunas, Klaipėda, Šiauliai and Panevėžys. Lithuanians who are the titular nation and form the majority of the country's population, belong to the ethnolinguistic group of Balts and speak Lithuanian. For millennia, the southeastern shores of the Baltic Sea were inhabited by various Baltic tribes. In the 1230s, Lithuanian lands were united for the first time by Mindaugas, who formed the Kingdom of Lithuania on 6 July 1253. Subsequent expansion and consolidation resulted in the Grand Duchy of Lithuania, which by the 14th century was the largest country in Europe. In 1386, the Grand Duchy entered into a de facto personal union with the Crown of the Kingdom of Poland. The two realms were united into the bi-confederal Polish-Lithuanian Commonwealth in 1569, forming one of the largest and most prosperous states in Europe. The Commonwealth lasted more than two centuries, until neighbouring countries gradually dismantled it between 1772 and 1795, with the Russian Empire annexing most of Lithuania's territory. Towards the end of World War I, Lithuania declared independence in 1918, founding the modern Republic of Lithuania. In World War II, Lithuania was occupied by the Soviet Union, then by Nazi Germany, before being reoccupied by the Soviets in 1944. Lithuanian armed resistance to the Soviet occupation lasted until the early 1950s. On 11 March 1990, a year before the formal dissolution of the Soviet Union, Lithuania became the first Soviet republic to break away when it proclaimed the restoration of its independence. Lithuania is a developed country with a high-income and an advanced economy ranking very high in Human Development Index.'''
context2 = 'Lithuania ranks highly in digital infrastructure,[25][26] press freedom and happiness.[27] Lithuania is a member of the United Nations, the European Union, the Council of Europe, the Council of the Baltic Sea States, the Eurozone, the Nordic Investment Bank, the International Monetary Fund, the Schengen Agreement, NATO, OECD and the World Trade Organization. It also participates in the Nordic-Baltic Eight (NB8) regional co-operation format.'

question = 'When Lithuania declared its independence?'
question = 'What is an area of Lithuania?'
question = 'What is a population of Lithuania?'
# question = 'What is a capital of Lithuania?'
text = f'''
[CLS] {context.strip()} [SEP] {question} [SEP]: 
'''
text = text.strip()

In [32]:
toks_in = tkz(text).input_ids
toks_in = torch.tensor(toks_in, dtype=torch.long, device=device)
toks_out = model.gen_on_wiki(toks_in)
s_out = tkz.decode(toks_out.squeeze())
s_out

'[CLS] 2. 89 million [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [33]:
len(toks_in), len(toks_out)

(472, 1)